# W8: Advanced Data Visualization
- Contributer: Dr. Zhonghua Zheng, Yuan Sun
- Course Unit: Earth and Environmental Data Science (EART60702)
- Last modified date: 07 April, 2024

## Intended Learning Outcomes (ILOs)
- Data visualization with Cartopy: develop advanced skills in visualizing geospatial data using specialized tools such as Cartopy, and learn how to create interactive and informative maps that effectively communicate complex spatial patterns and relationships.
- Geospatial visualization: understand cartographic principles and techniques, including map projection, coordinate systems, symbolization, and thematic mapping.

## 1. Cartopy
- Cartopy is a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses. Cartopy document: https://scitools.org.uk/cartopy/docs/latest/getting_started/index.html.
- Key features of cartopy are its object oriented projection definitions, and its ability to transform points, lines, vectors, polygons and images between those projections.
- Cartopy has exposed an interface to enable easy map creation using matplotlib.
- Cartopy Installation: https://scitools.org.uk/cartopy/docs/latest/installing.html

In [ ]:
import cartopy.crs as ccrs # import projections
import cartopy.feature as cf # import features
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

### 1.1 Basemap and mapping (40 mins)
- visualize `./data/W8.nc/`: https://github.com/m-edal/Earth-Env-DS-MSc-Course/blob/main/labs/data/W8.nc

The original data comes from https://figshare.com/collections/A_global_dataset_of_air_temperature_derived_from_satellite_remote_sensing_and_weather_stations/4081802
```Python
raw_data = xr.open_dataset('Tair_SG_2005.nc')
raw_data_1yr = raw_data.sel('time' = '1970-01-01 00:00:00')
raw_data_1yr.to_netcdf('W8.nc')
```

In [ ]:
# Creating a basic map is as simple as telling Matplotlib to use a specific map projection, and then adding some coastlines to the axes
ax = plt.axes(projection=ccrs.PlateCarree()) # specify projection
ax.coastlines() # add coastlines
ax.stock_img() # add an underlay image to the map
lon_min = -180
lon_max = 180
lat_min = -90
lat_max = 90
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.get_extent() # Get the extent (x0, x1, y0, y1) of the map in the given coordinate system
# ax.set_xticks(range(lon_min, lon_max + 1, 90)) # add xticks
ax.set_xticks([-180, -90, 0, 90, 180], crs=ccrs.PlateCarree()) 
ax.set_yticks([-90, -45, 0, 45, 90], crs=ccrs.PlateCarree())

plt.show()

In [ ]:
labelcolor = '#05101f'
labelfont = 8
ax2 = plt.axes(projection=ccrs.Mollweide()) 
ax2.add_feature(cf.LAND)
ax2.add_feature(cf.RIVERS)
ax2.add_feature(cf.LAKES)
ax2.coastlines(edgecolor=labelcolor, alpha = 0.2)
ax2.add_feature(cf.BORDERS, edgecolor=labelcolor, alpha = 0.2)
grid = ax2.gridlines(draw_labels=True, x_inline=False, y_inline=False, linestyle='--')
grid.xlabel_style = {'color': labelcolor, 'fontsize': labelfont}
grid.ylabel_style = {'color': labelcolor, 'fontsize': labelfont}

plt.show()

In [ ]:
ds = xr.open_dataset('./data/W8.nc') # global data
ds

In [ ]:
ax3 = plt.axes(projection=ccrs.Mollweide()) 
ax3.coastlines(edgecolor=labelcolor, alpha = 0.2)
ax3.add_feature(cf.BORDERS, edgecolor=labelcolor, alpha = 0.2)
grid = ax3.gridlines(draw_labels=True, x_inline=False, y_inline=False, linestyle='--')
grid.xlabel_style = {'color': labelcolor, 'fontsize': labelfont}
grid.ylabel_style = {'color': labelcolor, 'fontsize': labelfont}
mesh = ax3.pcolormesh(ds.lon, ds.lat, ds.Tair, transform=ccrs.PlateCarree(), cmap='viridis')

plt.show()

**Q1: add a data mask to the grid cells where temperature is higher than 295K.**

In [ ]:
ax3 = plt.axes(projection=ccrs.Mollweide()) 
ax3.coastlines(edgecolor=labelcolor, alpha = 0.2)
ax3.add_feature(cf.BORDERS, edgecolor=labelcolor, alpha = 0.2)
grid = ax3.gridlines(draw_labels=True, x_inline=False, y_inline=False, linestyle='--')
grid.xlabel_style = {'color': labelcolor, 'fontsize': labelfont}
grid.ylabel_style = {'color': labelcolor, 'fontsize': labelfont}
mesh = ax3.pcolormesh(ds.lon, ds.lat, 
                      np.ma.masked_where(ds.Tair > 295, ds.Tair), 
                      transform=ccrs.PlateCarree(), cmap='viridis')

plt.show()

**Q2: please visualize './data/W6.nc/': `https://github.com/m-edal/Earth-Env-DS-MSc-Course/blob/main/labs/data/W6.nc` through 2 ways: `pcolormesh()` and `contourf()`**
- `pcolormesh()`: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pcolormesh.html
- `contourf()`: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contourf.html

In [ ]:
ds = xr.open_dataset('./data/W6.nc') # regional data
ds

In [ ]:
# pcolormesh()


In [ ]:
# contourf()


**Q3: please use the following commands to do the visualization for the UK**
```Python
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
                                                name='admin_1_states_provinces',
                                                scale='10m',
                                                facecolor='none')
```

In [ ]:
import cartopy.feature as cfeature

fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# natural earth
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
                                                name='admin_1_states_provinces',
                                                scale='10m',
                                                facecolor='none')
# please add the extent for the UK
ax.set_extent([], ccrs.PlateCarree())

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)

# please add a line below

plt.show()

### 1.2 Open Street Map visualization using Cartopy (20 mins)

In [ ]:
import cartopy.io.img_tiles as cimgt # 'img_tiles' containes functions handlling image tiles for map plotting

request = cimgt.OSM()
extent = [-2.275, -2.2186, 53.47, 53.50] # set extend of the figure [lon_min, lon_max, lat_min, lat_max]

ax = plt.axes(projection=request.crs)
ax.set_extent(extent)

ax.add_image(request, 13) # 13 is the zoom level
plt.show()

In [ ]:
# mapping by a center point (latitude, longitude)
center_lat = 53.4668
center_lon = -2.2339

# Width and height of the map in degrees
width_deg = 0.0025
height_deg = 0.001

# Calculate extent based on center point, width, and height
min_lon = center_lon - (width_deg / 2)
max_lon = center_lon + (width_deg / 2)
min_lat = center_lat - (height_deg / 2)
max_lat = center_lat + (height_deg / 2)
extent = [min_lon, max_lon, min_lat, max_lat]

plt.figure(figsize=(8, 6))
# Request map tiles
req = cimgt.OSM()
ax = plt.axes(projection=req.crs)
ax.set_extent(extent)
ax.add_image(req, 19)  # Adjust zoom level as needed
plt.show()

# NOTE: zoom specifications should be selected based on extent:
# -- 2     = coarse image, select for worldwide or continental scales
# -- 4-6   = medium coarseness, select for countries and larger states
# -- 6-10  = medium fineness, select for smaller states, regions, and cities
# -- 10-12 = fine image, select for city boundaries and zip codes
# -- 14+   = extremely fine image, select for roads, blocks, buildings

Other practises: 
- https://www.theurbanist.com.au/2021/03/plotting-openstreetmap-images-with-cartopy/
- https://makersportal.com/blog/2020/4/24/geographic-visualizations-in-python-with-cartopy